In [ ]:
import logging
import os
import sys
import textwrap
from typing import Generator, Iterator, Literal, Union

import httpx
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import openai
import pandas as pd
import seaborn as sns
import torch
from pydantic import BaseModel
import scipy as sp

from llama_index.core import (
    Settings,
    SimpleDirectoryReader,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.core.base.llms.types import ChatMessage
from llama_index.core.chat_engine import CondensePlusContextChatEngine
from llama_index.core.chat_engine.types import BaseChatEngine
from llama_index.core.indices.base import BaseIndex
from llama_index.core.node_parser import HierarchicalNodeParser, MarkdownNodeParser
from llama_index.core.storage import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai_like import OpenAILike

In [ ]:
questions = [
    "O que é um laço?",
    "O que são algoritmos?",
    "Qual a diferença entre while e do-while?",
    "Meu código não está funcionando.",
    "O que são métodos? Como os utilizo?",
    "Qual é a vantagem de usar métodos em programação?",
    "O que é um erro de compilação?",
    "Por que meu programa está travando?",
    "Por que meu programa dá o resultado errado mesmo compilando?",
    "Para que serve a classe Scanner?",
    "Como eu imprimo na tela?",
    "O que são ifs aninhados?",
    "Como eu escrevo um algoritmo para resolver um problema?",
    "O que é uma classe?",
    "Como eu crio e uso classes no meu programa?",
    "O que é um array?",
    "O que é uma rede neural?",
    "O que é uma variável e como eu a declaro?",
    "Qual a diferença entre uma variável local e uma global?",
    "Como que eu leio um arquivo?",
    "Para que serve o comando switch-case?",
    "O que é um loop infinito?",
    "Como eu faço para tratar exceções?",
    "Qual a diferença entre while e for?",
    "Como eu faço para conectar ao banco de dados?",
    "O que é polimorfismo em programação orientada a objetos?",
    "Como eu faço para debugar meu código no BlueJ?",
    "O que é um framework?",
    "Como eu faço para usar funções recursivas?",
    "Como eu faço para ordenar um vetor?",
    "O que são atributos em uma classe?",
    "Como eu faço para criar uma interface gráfica em Java?",
    "O que é encapsulamento?",
    "Qual a diferença entre memória heap e stack?",
    "Como eu faço para implementar herança?",
    "O que é pensamento computacional?",
    "O que é um algoritmo de busca?",
    "Qual a utilidade da tabela verdade?",
    "O que é uma função?",
    "Como eu utilizo operadores aritméticos?",
    "O que é uma constante?",
    "Qual a diferença entre um inteiro e um float?",
    "O que são operadores lógicos?",
    "Como eu faço para converter tipos de dados?",
    "O que é um ponteiro?",
    "Como eu faço para manipular strings?",
    "O que são estruturas de controle de fluxo?",
    "Qual a diferença entre um vetor e uma matriz?",
    "Como eu faço para iterar sobre uma lista?",
    "O que é um módulo?",
    "Como eu faço para usar bibliotecas externas?",
    "O que é um comentário?",
    "Qual a diferença entre compilar e interpretar um programa?",
    "Como eu faço para medir o tempo de execução de um código?",
    "O que são parâmetros e argumentos em funções?",
    "Como eu faço para depurar um erro de lógica?",
    "O que são padrões de projeto?",
    "O que é teste de mesa?",
    "O que é teste de mesa?",
]

## Setup do host

No Brev.dev, as máquinas tem que ser configuradas com SSH e Ollama da seguinte forma.

### Ollama

Ele vai reclamar. Não tem problema.

```shell
$ sudo curl -L https://ollama.com/download/ollama-linux-amd64 -o /usr/bin/ollama
$ sudo chmod +x /usr/bin/ollama
$ OLLAMA_HOST=0.0.0.0 OLLAMA_NUM_PARALLEL=2 OLLAMA_KEEP_ALIVE=2h ollama serve &
$ ollama pull llama3:70b-instruct-q8_0
```

### SSH

Na tua máquina, com o Brev instalado:

```shell
$ brev port-forward $BREV_HOST -p 11435:11434
```

Nesse caso, `$BREV_HOST` é `tcc`.

In [ ]:
ollama = openai.AsyncOpenAI(base_url="http://localhost:11435/v1", api_key="ollama")

In [ ]:
stream = await ollama.chat.completions.create(
    stream=True,
    model="llama3:70b-instruct-q8_0",
    messages=[
        {
            "role": "user",
            "content": "hi",
        }
    ],
)

async for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

In [ ]:
llm = openai.AsyncOpenAI(base_url="http://10.88.190.244:9099", api_key="0p3n-w3bu!")

In [ ]:
responses = []
for q in questions:
    for _ in range(100):
        stream = await llm.chat.completions.create(
            stream=True,
            model="llama_index_retrieval_pipeline",
            messages=[
                {
                    "role": "user",
                    "content": q,
                }
            ],
        )

        resp = ""
        async for chunk in stream:
            resp += chunk.choices[0].delta.content or ""

        responses.append({"query": q, "response": resp})

In [ ]:
scores: list[dict[str, str | int]] = list()

for qr in responses:
    q = qr["query"]
    resp = qr["response"]

    stream = await ollama.chat.completions.create(
        stream=True,
        model="llama3:70b-instruct-q8_0",
        messages=[
            {
                "role": "system",
                "content": "Analyze the following exchange and give it a grade from 0 to 10. Only respond with a number.",
            },
            {"role": "user", "content": q},
            {"role": "system", "content": resp},
        ],
    )

    while True:
        quality = ""
        async for chunk in stream:
            quality += chunk.choices[0].delta.content or ""

        try:
            scores.append({"query": q, "response": resp, "score": int(quality)})
            break
        except ValueError as e:
            print(e)
            continue

In [ ]:
if os.path.exists("queries.csv"):
    queries_pd = pd.read_csv("queries.csv")
else:
    queries_pd = pd.DataFrame(scores, columns=["query", "response", "score"])
    queries_pd.to_csv("queries.csv", index=False)

In [ ]:
queries_pd

In [ ]:
sorted_df = queries_pd.sort_values(by="score", ascending=False)

In [ ]:
described_df = sorted_df.groupby("query")["score"].describe().sort_values(by="mean", ascending=False)

In [ ]:
z_score = sp.stats.norm.ppf(0.975)  # 95% confidence interval
described_df["CI_lower"] = described_df["mean"] - z_score * (described_df["std"] / (described_df["count"] ** 0.5))
described_df["CI_upper"] = described_df["mean"] + z_score * (described_df["std"] / (described_df["count"] ** 0.5))

In [ ]:
described_df

In [ ]:
import math


def square_wrap(text, width):
    # # Estimate the width based on the square root of the length of the text
    # ideal_width = int(math.sqrt(len(text)))

    # # Try to find the best width within the tolerance range
    # for w in range(width - ideal_width, width + ideal_width + 1):
    #     wrapped_text = textwrap.wrap(text, width=w)
    #     if len(wrapped_text) * max(len(line) for line in wrapped_text) < len(text) * 1.5:
    #         return '\n'.join(wrapped_text)

    # # Fallback to the estimated width if no better width is found within tolerance
    # print('failure')
    # return '\n'.join(textwrap.wrap(text, width=width))

    # Calculate the ideal width based on the square root of the length of the text
    ideal_width = max(width, int(math.sqrt(len(text))))

    # Initialize variables to track the best configuration
    min_diff = float("inf")  # Large initial difference
    best_wrap = None

    # Explore possible widths around the ideal width
    for w in range(ideal_width - 5, ideal_width + 5 + 1):
        # Wrap text at current width
        wrapped_text = textwrap.wrap(text, width=w)

        # Calculate the "squareness" by comparing the number of lines to the ideal width
        current_diff = abs(len(wrapped_text) - w)

        # Update the best found configuration if this one is closer to a square
        if current_diff < min_diff:
            min_diff = current_diff
            best_wrap = wrapped_text

    # Check if a wrapping configuration was found
    if best_wrap:
        return "\n".join(best_wrap)
    else:
        # Fallback if no wrapping configuration is close to square
        print("failure")
        return "\n".join(textwrap.wrap(text, width=ideal_width))

In [ ]:
sub_df = described_df.sample(frac=0.16)
sub_df = sub_df.sort_values(by="mean", ascending=False)
sub_df

In [ ]:
qs = ["O que é um laço?",
"O que são algoritmos?",
"Meu código não está funcionando.",
"O que são métodos? Como os utilizo?",
"Qual é a vantagem de usar métodos em programação?",
"O que é um erro de compilação?",
"Por que meu programa está travando?",
"Por que meu programa dá o resultado errado mesmo compilando?",
"Para que serve a classe Scanner?",
"Como eu imprimo na tela?",
"O que são ifs aninhados?",
"Como eu escrevo um algoritmo para resolver um problema?",
"O que é uma classe?",
"Como eu crio e uso classes no meu programa?",
"O que é um array?",
"O que é uma rede neural?",]

sub_df = described_df.loc[qs].sort_values(by="mean", ascending=False)

In [ ]:
plt.rcParams.update({"font.size": 13})

plt.figure(figsize=(12, 10))
ax = plt.gca()

errors = sub_df["max"] - sub_df["min"]
sub_df["error_lower"] = sub_df["mean"] - sub_df["CI_lower"]
sub_df["error_upper"] = sub_df["CI_upper"] - sub_df["mean"]

plt.grid(axis="x", linestyle="--", linewidth=0.5)
ax.set_xlim(0, 10.5)

ax.barh(sub_df.index, sub_df["mean"], color="skyblue", zorder=2, height=0.65)
ax.scatter(sub_df["min"], sub_df.index, color="orange", marker="D", zorder=5, label="Mínimo")
ax.scatter(sub_df["max"], sub_df.index, color="green", marker="D", zorder=6, label="Máximo")
ax.errorbar(
    sub_df["mean"],
    sub_df.index,
    xerr=sub_df["std"],
    fmt="none",
    ecolor="gray",
    capsize=5,
    capthick=2,
    label="Desvio padrão",
    elinewidth=2,
    zorder=3,
)
ax.errorbar(
    sub_df["mean"],
    sub_df.index,
    xerr=[sub_df["error_lower"], sub_df["error_upper"]],
    fmt="none",
    ecolor="red",
    capsize=0,
    label="Intervalo de confiança (95%)",
    elinewidth=5,
    zorder=4,
)

ax.invert_yaxis()
ax.legend(loc="best")

width = 40
labels = [square_wrap(label.get_text(), width) for label in ax.get_yticklabels()]

ax.set_yticks(range(len(sub_df.index)))

# ax.set_yticks(ax.get_yticks())
ax.set_yticklabels(labels, rotation=0)

plt.xlabel("Escore médio")
plt.title("Avaliação automática de respostas de perguntas sobre programação")
plt.tight_layout()
plt.savefig("scores2.png", dpi=300)
plt.savefig("scores2.pdf", backend="pgf")
plt.savefig("/home/debem/personal/pucrs/tcc/cur/monografia/thesis/media/scores2.pdf", backend="pgf")
plt.show()

In [ ]:
with pd.option_context("max_colwidth", 1000):
    print(described_df.to_latex(index=True, float_format="%.2f", longtable=False))

In [ ]:
fig = plt.figure(figsize=(9, 9))
ax = fig.gca()
_df = described_df

errors = _df["max"] - _df["min"]

ax.grid(axis="x", linestyle="--", linewidth=0.5)
ax.set_xlim(0, 10.5)

ax.barh(_df.index, _df["mean"], color="skyblue", zorder=2)
ax.errorbar(
    _df["min"],
    _df.index,
    xerr=[np.zeros(len(errors)), errors],
    fmt="none",
    ecolor="gray",
    capsize=5,
    label="Variação do score",
)
ax.invert_yaxis()
ax.set_xlabel("Score médio")

plt.tight_layout()
plt.savefig("/home/debem/personal/pucrs/tcc/cur/monografia/thesis/media/scoresall.pgf")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

part_size = len(described_df) // 2
remainder = len(described_df) % 2

data1 = described_df[:part_size]
data2 = described_df[part_size : 2 * part_size]
data3 = described_df[2 * part_size : 3 * part_size]
data4 = described_df[3 * part_size : 4 * part_size + remainder]

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 9))

for ax, _df in zip([ax1, ax2, ax3, ax4], [data1, data2, data3, data4]):
    errors = _df["max"] - _df["min"]

    ax.grid(axis="x", linestyle="--", linewidth=0.5)
    ax.set_xlim(0, 10.5)

    ax.barh(_df.index, _df["mean"], color="skyblue", zorder=2)
    ax.errorbar(
        _df["min"],
        _df.index,
        xerr=[np.zeros(len(errors)), errors],
        fmt="none",
        ecolor="gray",
        capsize=5,
        label="Variação do score",
    )
    ax.invert_yaxis()
    ax.set_xlabel("Score médio")

# ax1.set_yticklabels(fontsize=12)

plt.tight_layout()
plt.savefig("scoresall.png", dpi=400)
plt.show()

In [ ]:
for __df in [described_df[: len(described_df) // 2], described_df[len(described_df) // 2 :]]:
    part_size = len(__df) // 2
    remainder = len(__df) % 2

    data1 = __df[:part_size]
    data2 = __df[part_size : 2 * part_size + remainder]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(25, 10))

    for ax, _df in zip([ax1, ax2], [data1, data2]):
        errors = _df["max"] - _df["min"]

        ax.grid(axis="x", linestyle="--", linewidth=0.5)
        ax.set_xlim(0, 10.5)

        ax.barh(_df.index, _df["mean"], color="skyblue", zorder=2)
        ax.errorbar(
            _df["min"],
            _df.index,
            xerr=[np.zeros(len(errors)), errors],
            fmt="none",
            ecolor="gray",
            capsize=5,
            label="Variação do score",
        )
        ax.invert_yaxis()
        ax.set_xlabel("Score médio")

        ax.tick_params("y", labelsize=15)

    plt.tight_layout()
    plt.savefig(f"scoresall_{__df.iloc[0].name[0:3]}.png", dpi=400)
    plt.show()